# Проект для «Викишоп» с BERT


Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#SGDClassifier" data-toc-modified-id="SGDClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>SGDClassifier</a></span></li><li><span><a href="#Дамми-модель" data-toc-modified-id="Дамми-модель-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Дамми модель</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Подготовка

Импортируем библиотеки

In [1]:
# !pip install transformers

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import notebook

import torch
import transformers

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import time

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# from google.colab import drive
# drive.mount ('/content/drive')

Загружаем и осматриваем данные
Колонка `Unnamed: 0` нам не нужна, тем более, что данные там собраны не по порядку, судя по индексам

In [3]:
# !ls /content/drive/MyDrive/

In [4]:
try:
    data = pd.read_csv('/content/drive/MyDrive/datasets/toxic_comments.csv', usecols = ['text','toxic'])
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', usecols = ['text','toxic'])
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


Перед нами стоит задача классификации, которую мы будем решать с помощью очень требовательной к ресурсам модели. 159292 комментария модель будет обрабатывать целый день, возьмем тренировочную выборку в 12000 штук и тестовую в 3000 штук.

Используемая модель

[unitary/toxic-bert](https://huggingface.co/unitary/toxic-bert)


Посмотрим на дисбаланс классов

In [5]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Возьмем выборку в 12000 + 3000 = 15000 из этого датасета.

In [8]:
data_test = data.sample(3000, random_state = 12345)
data_test.head()

,text,toxic
109486,Expert Categorizers \n\nWhy is there no menti...,0
104980,"""\n\n Noise \n\nfart* talk. """,1
82166,"An indefinite block is appropriate, even for a...",0
18721,I don't understand why we have a screenshot of...,0
128178,"Hello! Some of the people, places or things yo...",0


In [9]:
data_train = data[~data.index.isin(data_test.index)].sample(12000, random_state = 12345)
data_train.head()

,text,toxic
120742,he touched me in a naughty place,1
84961,"-Russell29 2 April 2007, 16:26 (UTC)",0
93329,I'm not sure but I think I have read that Brit...,0
47628,"""early 2004, before or after brendon's joining...",0
26917,"""\n\n ... isn't that violating a block? (Is no...",0


In [10]:
data_train['toxic'].value_counts()

0    10795
1     1205
Name: toxic, dtype: int64

In [11]:
data_test['toxic'].value_counts()

0    2677
1     323
Name: toxic, dtype: int64

В тренировочной выборке данные сбалансированы, в тестовой баланс изначальный. Сбросим индексы

Теперь токенизируем комментарии

In [12]:
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')

tokenized_train = data_train['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens = True, truncation = True))

tokenized_train

120742    [101, 2002, 5028, 2033, 1999, 1037, 20355, 217...
84961     [101, 1011, 5735, 24594, 1016, 2258, 2289, 101...
93329     [101, 1045, 1005, 1049, 2025, 2469, 2021, 1045...
47628     [101, 1000, 2220, 2432, 1010, 2077, 2030, 2044...
26917     [101, 1000, 1012, 1012, 1012, 3475, 1005, 1056...
                                ...                        
72602     [101, 2004, 2521, 2004, 1045, 3342, 1010, 1045...
5204      [101, 2417, 7442, 6593, 2831, 1024, 9166, 9019...
53622     [101, 2748, 1010, 2002, 2003, 2157, 1012, 4409...
120089    [101, 1045, 2228, 2848, 2038, 2445, 2019, 6581...
51487     [101, 2061, 2017, 2024, 3038, 1037, 14686, 104...
Name: text, Length: 12000, dtype: object

Слишком больших комментариев не так много. Уберем комменты с более чем 512 токенами

In [13]:
data_train = data_train.reset_index(drop = True)

In [14]:
max_len_train = tokenized_train.apply(len).max()
max_len_train

512

Добавим отступы

In [15]:
padded_train = np.array([i + [0] * (max_len_train - len(i)) for i in tokenized_train.values])
padded_train

array([[ 101, 2002, 5028, ...,    0,    0,    0],
       [ 101, 1011, 5735, ...,    0,    0,    0],
       [ 101, 1045, 1005, ...,    0,    0,    0],
       ...,
       [ 101, 2748, 1010, ...,    0,    0,    0],
       [ 101, 1045, 2228, ...,    0,    0,    0],
       [ 101, 2061, 2017, ...,    0,    0,    0]])

Добавим маску важности

In [16]:
attention_mask_train = np.where(padded_train != 0, 1, 0)
attention_mask_train

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

Инициализируем обученную модель, передаем ей конфигурацию

In [17]:
model = transformers.BertModel.from_pretrained('unitary/toxic-bert')

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Преобразуем тексты в эмбеддинги. Наливаем чаек, включаем кино.


Важно:
`В процессе обработки данных через несколько минут происходит краш системы, чтобы этого не происходило необходимо отключить разгон ядер процессора, в том числе TurboBoost`

`Графическую карту не подключал`

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [19]:
batch_size = 100
embeddings_train = []
for i in notebook.tqdm(range(padded_train.shape[0] // batch_size)):
        batch = torch.LongTensor(padded_train[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask_train[batch_size*i:batch_size*(i+1)])
        
#         time.sleep(5)
        
        with torch.no_grad():
            batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))
        
#         time.sleep(5)
        
        embeddings_train.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/120 [00:00<?, ?it/s]

In [20]:
# x_train = np.load('wikishop_train_1.npy')

In [21]:
features_train = np.concatenate(embeddings_train)
features_train.shape


(12000, 768)

12000 объектов по 768 фичей в каждом. Признаки трейна готовы.
Проделаем то же самое с тестовой выборкой. 

In [22]:
tokenized_test = data_test['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens = True, truncation = True))


data_test = data_test.reset_index(drop = True)

max_len_test = tokenized_test.apply(len).max()

padded_test = np.array([i + [0] * (max_len_test - len(i)) for i in tokenized_test.values])

attention_mask_test = np.where(padded_test != 0, 1, 0)

In [23]:
batch_size = 100
embeddings_test = []
for i in notebook.tqdm(range(padded_test.shape[0] // batch_size)):
        batch = torch.LongTensor(padded_test[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask_test[batch_size*i:batch_size*(i+1)])
        
#         time.sleep(5)
        
        with torch.no_grad():
            batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))
        
#         time.sleep(5)
        
        embeddings_test.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/30 [00:00<?, ?it/s]

In [24]:
# x_test = np.load('wikishop_test.npy')

In [25]:
features_test = np.concatenate(embeddings_test)
features_test.shape

(3000, 768)

Поделим тренировочную и тестовую выборки

In [26]:
x_train = features_train

In [27]:
y_train = data_train[0:12000]['toxic']

In [28]:
x_test = features_test

In [29]:
y_test = data_test[0:3000]['toxic']

## Обучение

### Логистическая регрессия

Подберем параметры для логистической регрессии с помощью `RandomizedSearchCV`

In [30]:
model_lr = LogisticRegression(max_iter=1000)

params = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10], 'class_weight':['balanced',None]}

In [31]:
grid_search_lr = GridSearchCV(estimator = model_lr,
                              param_grid = params,
                              cv = 3,
                              n_jobs = -1,
                              verbose = 0,
                              scoring = 'f1')

In [32]:
grid_search_lr.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10],
                         'class_weight': ['balanced', None]},
             scoring='f1')

In [33]:
grid_search_lr.best_score_

0.9535313306656241

Отличная метрика на кросс-валидации, попробуем другие модели

Лучшие параметры

In [34]:
grid_search_lr.best_params_

{'C': 0.1, 'class_weight': None}

### Решающее дерево

Воспользуемся `RandomizedSearchCV`

In [36]:
model_dt = DecisionTreeClassifier(random_state=12345)

params = {'max_depth': range(1,15), 'min_samples_split': range(2,10)}

In [37]:
grid_model_dt = RandomizedSearchCV(estimator = model_dt,
                                   param_distributions = params,
                                   n_iter = 20,
                                   cv = 3,
                                   n_jobs = -1,
                                   verbose = 0,
                                   scoring = 'f1')

In [38]:
grid_model_dt.fit(x_train, y_train)

RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=12345),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_depth': range(1, 15),
                                        'min_samples_split': range(2, 10)},
                   scoring='f1')

In [39]:
grid_model_dt.best_score_

0.9360769468573745

Метрика на кросс-валидации несколько хуже чем у логистической регрессии

### SGDClassifier

In [40]:
model_sgd = SGDClassifier(random_state=12345, max_iter=1000)

params = {'alpha': [0.0001, 0.00001, 0.000001],
          'penalty': ['l1','l2']}

In [41]:
grid_model_sgd = GridSearchCV(estimator = model_sgd,
                              param_grid = params,
                              cv = 3,
                              n_jobs = -1,
                              verbose = 0,
                              scoring = 'f1')

In [42]:
grid_model_sgd.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=SGDClassifier(random_state=12345), n_jobs=-1,
             param_grid={'alpha': [0.0001, 1e-05, 1e-06],
                         'penalty': ['l1', 'l2']},
             scoring='f1')

In [43]:
grid_model_sgd.best_score_

0.9421428518475597

Метрика также уступает логистической регрессии

### Дамми модель

Несмотря на высокую метрику, проверим на адекватность

In [44]:
model_dummy = DummyClassifier(random_state=12345)

params = {'strategy': ['prior', 'stratified']}

In [45]:
grid_model_dummy = GridSearchCV(estimator = model_dummy,
                                param_grid = params,
                                cv = 3,
                                n_jobs = -1,
                                verbose = 0,
                                scoring = 'f1')

In [46]:
grid_model_dummy.fit(x_train,y_train)

GridSearchCV(cv=3, estimator=DummyClassifier(random_state=12345), n_jobs=-1,
             param_grid={'strategy': ['prior', 'stratified']}, scoring='f1')

In [47]:
grid_model_dummy.best_score_

0.09884713446402839

Тестирование лучшей модели

In [48]:
preds = grid_search_lr.predict(x_test)
print('f1_score = ', f1_score(y_test, preds))

f1_score =  0.9546165884194053


## Выводы

На моделях с использованием BERT, предварительно настроенной на выявление токсичных комментариев, метрика f1 достигает 95,5%